<a href="https://colab.research.google.com/github/vinayshanbhag/python_concepts/blob/master/Decorators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Python Decorators

Python decorators can be used to dynamically alter the functionality of a function without having to change the source code of the function being decorated.

The ```decorator``` function takes the original function to be decorated as input. The inner function ```decorated``` wraps additional functionality around the original function. The ```decorator``` then returns the ```decorated``` inner function 



In [0]:
def decorator(original):
  print(f"decorator called for {original.__name__}")
  def decorated():
    print(f"calling {original.__name__}:")
    val = original()
    print(f":{original.__name__} ended")
    return val
  return decorated

To put this to use, we define a function ```foo``` and then reassign ```foo``` to decorated version

In [30]:
def foo():
  """
  Docstring for function foo goes here
  """
  print("inside foo")
  return "foo"
foo = decorator(foo)

decorator called for foo


Calling ```foo```, we can see the decorated behavior

In [31]:
x = foo()
print(f"function foo returned: {x}")

calling foo:
inside foo
:foo ended
function foo returned: foo


Alternatively, this can be simplified with the @decorator syntax.

In [32]:
@decorator
def foo():
  """
  Docstring for function foo goes here
  """
  print("inside foo")
  return "foo"

decorator called for foo


Calling this function, reveals decorated behavior

In [33]:
y = foo()
print(f"function foo returned: {y}\n")

calling foo:
inside foo
:foo ended
function foo returned: foo



To handle function arguments, pass ```*args``` and ```**kwargs``` to the wrapper function

In [0]:
def decorator(original):
  print(f"decorator called for {original.__name__}")
  def decorated(*args, **kwargs):
    print(f"calling {original.__name__}:")
    val = original(*args,**kwargs)
    print(f":{original.__name__} ended")
    return val
  return decorated

In [35]:
@decorator
def foo(n):
  """
  Docstring for function foo goes here
  """
  print(f"inside foo - {n}")
  return "foo"

decorator called for foo


In [36]:
foo(10);

calling foo:
inside foo - 10
:foo ended


But there is a problem... 

```help()``` returns incorrect information

In [37]:
help(foo)

Help on function decorated in module __main__:

decorated(*args, **kwargs)



because the ```__doc__``` string is empty

In [0]:
foo.__doc__

and ```__name__``` is that of the wrapper function

In [39]:
foo.__name__

'decorated'

To fix this, we use ```@wraps``` decorator from ```functools```

In [0]:
from functools import wraps
def decorator(original):
  print(f"decorator called for {original.__name__}")
  @wraps(original)
  def decorated(*args, **kwargs):
    print(f"calling {original.__name__}:")
    val = original(*args,**kwargs)
    print(f":{original.__name__} ended")
    return val
  return decorated

In [41]:
@decorator
def foo(n):
  """
  Docstring for function foo goes here
  """
  print(f"inside foo - {n}")
  return "foo"

decorator called for foo


Decorated behavior works the same as before

In [43]:
foo(10);

calling foo:
inside foo - 10
:foo ended


But now the function name is reported correctly and ```help()``` works as expected

In [44]:
foo.__name__

'foo'

In [45]:
help(foo)

Help on function foo in module __main__:

foo(n)
    Docstring for function foo goes here



### Chaining multiple decorators

Here we apply ```time_it``` decorator to ```some_long_running_func``` and then also apply the ```logger``` decorator. The problem here is that the inner most decorator returns a wrapper func, so the logger sees ```wrapper``` instead of our original function


In [0]:
import time

def logger(f):
  @wraps(f)
  def wrapper(*args, **kwargs):
    print(f"calling {f.__name__}():")
    return f(*args, **kwargs)
  return wrapper

def time_it(f):
  @wraps(f)
  def wrapper(*args, **kwargs):
    s = time.perf_counter()
    v = f(*args, **kwargs)
    e = time.perf_counter()
    print(f"{f.__name__} took:{e-s:0.3f}s")
    return v
  return wrapper

In [0]:
@logger
@time_it
def some_long_running_func():
  """Some long running function"""
  time.sleep(2)

In [55]:
some_long_running_func()

calling some_long_running_func():
some_long_running_func took:2.002s


In [56]:
help(some_long_running_func)

Help on function some_long_running_func in module __main__:

some_long_running_func()
    Some long running function



### Decorators using classes

In [0]:
class Decorator_class():
  def __init__(self, func_to_be_decorated):
    self.func_to_be_decorated = func_to_be_decorated
  
  def __call__(self, *args, **kwargs):
    print(f"calling {self.func_to_be_decorated.__name__}:")
    val = self.func_to_be_decorated(*args, **kwargs)
    print(f":{self.func_to_be_decorated.__name__} ended")
    return val

In [0]:
@Decorator_class
def some_func(*args):
  print(f"some_func({','.join(list(args))}):")
  return ' '.join(list(args))

In [64]:
z = some_func("test")
print(f"some_func returned:{z}")

calling some_func:
some_func(test):
:some_func ended
some_func returned:test
